In [1]:
import requests
import datetime
import re
import base64

In [31]:
def readOnlyLine(fileName):
    with open(f'{fileName}') as f:
        return next(f)
    
def music_handler(f, o):
    current_line = next(f)
    if current_line[0] == "!":
        return current_line 
    
    trackLink = current_line.strip()
    parameters = {"url": trackLink, "format": "json"}
    website = re.search('\/\/(.+?)\/', trackLink).group(1)
    trackJson = requests.get(f'https://{website}/oembed', params = parameters).json()
    trackThumb = trackJson['thumbnail_url']
    trackTitle = trackJson['title']
    htmlBlock = f'<a href="{trackLink}"><div class="ach"> <img class="ach-img" src={trackThumb}></img>{trackTitle}</div></a>\n'
    o.write(htmlBlock)
    music_handler(f, o)
    
def playlist_handler(f, o):
    current_line = next(f)
    if current_line[0] == "!":
        return current_line 
    
    trackLink = current_line.strip()
    parameters = {"grant_type": "client_credentials", "client_id": readOnlyLine("spotifyID.txt"), "client_secret": readOnlyLine("spotifySecret.txt")}
    
    authJson = requests.post(f'https://accounts.spotify.com/api/token', 
                             params = parameters,
                             headers={
                                 'Content-Type': "application/x-www-form-urlencoded"
                             }).json()

    accessToken = authJson['access_token']
    playlistsJson = requests.get(f'https://api.spotify.com/v1/users/22kb7mgop3ckzk6wvv7h6qnqy/playlists',
                                 params = parameters,
                                 headers={
                                     'Authorization': f'Bearer {accessToken}',
                                     'Content-Type': "application/json"
                                 }).json()
    
    for items in playlistsJson['items']:
        playlistLink = f'https://open.spotify.com/playlist/{items["id"]}'
        try:
            imgSrc = items['images'][0]['url']
        except:
            continue
        playlistTitle = items['name']
        htmlBlock = f'<a href="{playlistLink}"><div class="ach"> <img class="ach-img" src={imgSrc}></img>{playlistTitle}</div></a>\n\n'
        o.write(htmlBlock)
    playlist_handler(f, o)
    
with open("data.txt", "r+") as f:
    output = open(f'output {str(datetime.datetime.now())}.txt', "x")
    line = next(f)
    #check for a playlist link under the playlist flag
    if(line == "!playlist\n"):
        line = playlist_handler(f, output)
    # check for repeated music links under music flags and projects under project flag.
    while line != "":
        if line == "!music\n":
            line = music_handler(f, output)
            continue # so that we can check for EOF after recursion
        # elif l == "!projects":
        line = ""
    output.close()
            
    
        